In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').appName('Climalyticsat').getOrCreate()

In [ ]:
spark

# Load Data

In [ ]:
# Pfade definieren
csv_path = "/home/jovyan/work/climate_all_stations.csv"
parquet_path = "/home/jovyan/work/parquet/climate_all_stations"

In [ ]:
# CSV laden
df_csv = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv(csv_path)

In [ ]:
# Kurzer Check
df_csv.printSchema()
df_csv.show(5)

In [ ]:
df = spark.read.parquet("/home/jovyan/work/parquet/climate_all_stations")
df.printSchema()
df.show(5)


In [ ]:
# Als Parquet speichern
df_csv.write.mode("overwrite").parquet(parquet_path)

In [ ]:
meta_path = "/home/jovyan/work/stations_metadata.csv"

stations_meta = spark.read.option("header", True).option("inferSchema", True).csv(meta_path)

# umbenennen für Join-Kompatibilität
stations_meta = stations_meta.withColumnRenamed("id", "station_id") \
                             .withColumnRenamed("Höhe [m]", "altitude")

stations_meta.select("station_id", "altitude").show(5)



In [ ]:
# Klimadaten laden (Parquet)
climate_df = spark.read.parquet("/home/jovyan/work/parquet/climate_all_stations")

climate_df.printSchema()

In [ ]:
# Nur relevante Spalten
climate_df = climate_df.select("station_id", "date", "tl_mittel")

# Join mit Höhenmetadaten
df_joined = climate_df.join(stations_meta, on="station_id")

# Kurzer Check
df_joined.select("station_id", "date", "tl_mittel", "altitude").show(5)


In [ ]:
from pyspark.sql.functions import when, year, avg, col

# Jahr extrahieren
df = df_joined.withColumn("year", year("date"))

# Höhenzonen definieren
df = df.withColumn(
    "elevation_zone",
    when(col("altitude") < 500, "0–499 m (Lowland)")
    .when(col("altitude") < 1000, "500–999 m (Upland)")
    .when(col("altitude") < 1500, "1000–1499 m (Lower Alps)")
    .when(col("altitude") < 2000, "1500–1999 m (Alpine)")
    .otherwise("2000+ m (High Alpine)")
)

# Temperaturmittel je Jahr & Höhenzone
df_avg = df.groupBy("year", "elevation_zone") \
           .agg(avg("tl_mittel").alias("mean_temp")) \
           .orderBy("year", "elevation_zone")

df_avg.show(10, truncate=False)


In [ ]:
stations_meta = stations_meta.withColumnRenamed("Bundesland", "bundesland")

# Mitteltemperatur nach Jahr, Höhenzone und Bundesland
df_avg = df.groupBy("year", "elevation_zone", "bundesland") \
           .agg(avg("tl_mittel").alias("mean_temp")) \
           .orderBy("elevation_zone", "bundesland", "year")

df_avg.show(10, truncate=False)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Daten nach Pandas konvertieren
df_pd = df_avg.toPandas()

# Jahreszahl als int (optional)
df_pd["year"] = df_pd["year"].astype(int)

# Nach Höhenzonen gruppieren
zones = df_pd["elevation_zone"].unique()

# Diagramm pro Höhenzone
for zone in sorted(zones):
    plt.figure(figsize=(10, 6))
    subset = df_pd[df_pd["elevation_zone"] == zone]

    for bundesland in subset["bundesland"].unique():
        region_data = subset[subset["bundesland"] == bundesland]
        plt.plot(region_data["year"], region_data["mean_temp"], label=bundesland)

    plt.title(f"Temperaturtrend – {zone}")
    plt.xlabel("Jahr")
    plt.ylabel("Durchschnittstemperatur (°C)")
    plt.legend(loc="upper left", fontsize="small")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
from pyspark.sql.functions import when

stations_meta_zones = stations_meta.withColumn(
    "elevation_zone",
    when(col("altitude") < 500, "0–499 m (Lowland)")
    .when(col("altitude") < 1000, "500–999 m (Upland)")
    .when(col("altitude") < 1500, "1000–1499 m (Lower Alps)")
    .when(col("altitude") < 2000, "1500–1999 m (Alpine)")
    .otherwise("2000+ m (High Alpine)")
)


In [ ]:
from pyspark.sql.functions import count, avg, min, max

station_stats = stations_meta_zones.groupBy("elevation_zone", "bundesland") \
    .agg(
        count("*").alias("num_stations"),
        avg("altitude").alias("avg_altitude"),
        min("altitude").alias("min_altitude"),
        max("altitude").alias("max_altitude")
    ) \
    .orderBy("elevation_zone", "bundesland")

station_stats.show(truncate=False)


In [ ]:
pd_stats = station_stats.toPandas()
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.barplot(data=pd_stats, x="elevation_zone", y="num_stations", hue="bundesland")
plt.title("Anzahl Wetterstationen pro Höhenzone und Bundesland")
plt.ylabel("Anzahl Stationen")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
import seaborn as sns

# Spark → Pandas
stations_pd = stations_meta_zones.select(
    "station_id", "altitude", "elevation_zone", "bundesland", "Stationsname"
).toPandas()


# Sortierbare Höhenzonen
zone_order = [
    "0–499 m (Lowland)",
    "500–999 m (Upland)",
    "1000–1499 m (Lower Alps)",
    "1500–1999 m (Alpine)",
    "2000+ m (High Alpine)"
]

# Kategorische Sortierung zuweisen
stations_pd["elevation_zone"] = stations_pd["elevation_zone"].astype(
    CategoricalDtype(categories=zone_order, ordered=True)
)

# Plot
plt.figure(figsize=(12, 6))
sns.stripplot(
    data=stations_pd,
    x="elevation_zone",
    y="altitude",
    hue="bundesland",
    dodge=True,
    jitter=0.3,
    alpha=0.8,
    marker="o"
)

plt.title("Höhenverteilung der Wetterstationen pro Höhenzone (farbig nach Bundesland)")
plt.xlabel("Höhenzone")
plt.ylabel("Höhe der Station (m)")
plt.xticks(rotation=15)
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(title="Bundesland", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


In [ ]:

# Nur alpine Höhenzonen
zone_order = ["1500–1999 m (Alpine)", "2000+ m (High Alpine)"]

stations_pd["elevation_zone"] = stations_pd["elevation_zone"].astype(
    CategoricalDtype(categories=zone_order, ordered=True)
)

stations_subset = stations_pd[stations_pd["elevation_zone"].isin(zone_order)].copy()

# Höchste Stationen je Höhenzone & Bundesland
top_stations = (
    stations_subset[stations_subset["altitude"].notna()]
    .sort_values("altitude", ascending=False)
    .groupby(["elevation_zone", "bundesland"], as_index=False)
    .first()
)

top_stations


In [ ]:
print(stations_pd.columns)


In [ ]:
stations_name_pd = stations_meta.select("station_id", "Stationsname").toPandas()
top_stations = top_stations.drop(columns=["Stationsname"], errors="ignore")
top_stations = top_stations.merge(stations_name_pd, on="station_id", how="left")

top_stations_clean = top_stations.dropna(subset=["elevation_zone", "altitude", "Stationsname"])

plt.figure(figsize=(12, 6))

sns.stripplot(
    data=stations_subset,
    x="elevation_zone",
    y="altitude",
    hue="bundesland",
    jitter=0.3,
    dodge=True,
    alpha=0.8,
    marker="o"
)

for i, row in top_stations_clean.iterrows():
    offset = 30 if i % 2 == 0 else -40
    plt.text(
        x=row["elevation_zone"],
        y=row["altitude"] + offset,
        s=row["Stationsname"],
        ha="center",
        fontsize=8,
        color="black"
    )

plt.title("Messstationen in alpinen Höhenzonen (mit höchster Station je Bundesland)")
plt.xlabel("Höhenzone")
plt.ylabel("Höhe der Station (m)")
plt.xticks(rotation=15)
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", title="Bundesland")
plt.tight_layout()
plt.show()